In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "MA"
BARK_VAR = "CA"
FIA_SPCD = 361

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(-0.01348, 0.98155, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,1475.000000,1.475000e+03,1475.0,1475.000000,1475.000000,1475.000000,1475.000000,1475.000000,1475.000000,1475.000000,...,1.0,1.00,1475.000000,1475.000000,1475.000000,1475.000000,1475.0,1475.000000,1475.000000,1475.000000
mean,584.055593,2.710690e+11,361.0,89.641673,0.455769,3.086251,23.991186,10.355051,0.359193,0.316061,...,0.1,0.35,2005.159322,8.148475,2015.159322,7.241356,10.0,1.810847,6.133559,163.991864
std,61.436465,1.719146e+11,0.0,42.283838,0.204544,1.837740,9.639742,6.137993,0.155091,0.148699,...,NaN,NaN,2.462412,2.179291,2.462412,2.112520,0.0,0.995317,5.839479,111.658613
min,503.000000,6.010155e+10,361.0,10.000000,0.000000,0.000000,1.000000,5.000000,0.010000,0.010000,...,0.1,0.35,2001.000000,1.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,510.000000,6.010979e+10,361.0,61.400759,0.300000,1.658063,17.000000,6.500000,0.250000,0.200000,...,0.1,0.35,2003.000000,6.000000,2013.000000,5.000000,10.0,1.000000,1.000000,71.000000
50%,611.000000,4.102020e+11,361.0,88.380000,0.450000,3.106686,25.000000,8.300000,0.350000,0.300000,...,0.1,0.35,2006.000000,9.000000,2016.000000,7.000000,10.0,1.000000,3.000000,132.000000
75%,611.000000,4.102030e+11,361.0,112.080000,0.630000,4.642576,31.000000,11.600000,0.450000,0.400000,...,0.1,0.35,2007.000000,10.000000,2017.000000,9.000000,10.0,3.000000,12.000000,268.000000
max,712.000000,5.306046e+11,361.0,327.817056,1.000000,6.283185,47.000000,50.900000,0.900000,0.900000,...,0.1,0.35,2009.000000,12.000000,2019.000000,11.000000,10.0,4.000000,18.000000,373.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [12:57<00:00,  2.57it/s, 255 steps of size 2.61e-02. a


Done.
Samples saved at ../../models/maicf/MA//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MA//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [20:42<00:00,  1.61it/s, 127 steps of size 2.71e-02. a


Done.
Samples saved at ../../models/maicf/MA//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MA//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/MA//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MA//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [13:01<00:00,  2.56it/s, 255 steps of size 2.46e-02. a


Done.
Samples saved at ../../models/maicf/MA//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MA//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [34:58<00:00,  1.05s/it, 511 steps of size 1.31e-02. a


Done.
Samples saved at ../../models/maicf/MA//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MA//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [09:08<00:00,  3.65it/s, 127 steps of size 2.72e-02. a


Done.
Samples saved at ../../models/maicf/MA//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MA//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [10:14<00:00,  3.26it/s, 255 steps of size 2.56e-02. a


Done.
Samples saved at ../../models/maicf/MA//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MA//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [16:48<00:00,  1.98it/s, 255 steps of size 2.41e-02. a


Done.
Samples saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [17:01<00:00,  1.96it/s, 255 steps of size 2.31e-02. a


Done.
Samples saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MA//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
